In [42]:
# Célula 1: Imports e funções utilitárias
import json
from gurobipy import Model, GRB, quicksum
import math

def carregar_json(caminho_arquivo):
    with open(caminho_arquivo, 'r') as f:
        data = json.load(f)
    return data

# Célula 2: Função de construção do modelo
def constroi_modelo(json_data):
    """Constrói o modelo HSTT a partir dos dados carregados do JSON."""
    # Dados do JSON
    D = json_data["D"]               # Dias
    P = json_data["P"]               # Períodos
    T = json_data["T"]               # Professores
    C = json_data["C"]               # Turmas
    E = json_data["E"]               # Eventos
    R = json_data["R"]               # Carga horária de cada evento
    L = json_data["L"]               # Limite diário de aulas por evento
    M = json_data["M"]               # Número mínimo de aulas duplas por evento
    gamma = json_data["gamma"]       # Custo dia trabalhado por professor
    omega = json_data["omega"]       # Custo intervalo ocioso
    delta = json_data["delta"]       # Custo aula dupla não atendida
    # Mapeamento de professor para evento
    teacher_of_event = json_data["teacher_of_event"]
    # Mapeamento de turma para evento
    class_of_event = json_data["class_of_event"]
    V = json_data["V"]               # Disponibilidade dos professores

    # Conjuntos derivados
    Et = {t: [e for e in E if teacher_of_event[e] == t]
          for t in T}  # Eventos por professor
    Ec = {c: [e for e in E if class_of_event[e] == c]
          for c in C}    # Eventos por turma

    # SGe – slots onde e pode iniciar aula dupla
    SGe = {}
    for e in E:
        SGe[e] = []
        for d in D:
            for p in P[:-1]:  # Excluindo o último período
                if e in V and d in V[e] and p in V[e][d] and (p+1) in V[e][d] and V[e][d][p] and V[e][d][p+1]:
                    # Verificando se o evento e, dia d, e períodos p e p+1 existem e são válidos
                    SGe[e].append((d, p))

    # U e Q – arcos para o grafo de períodos ociosos
    U, Q = [], []
    for m in P[:-2]:  # Excluindo os dois últimos períodos
        for n in P:
            if n >= m:
                Q.append((m, n))
            if n >= m + 2:
                U.append((m, n))

    # Construção do modelo
    modelo = Model("HSTT_Model")

    # ------------------------------- Variáveis
    # Variáveis de alocação de eventos
    x = modelo.addVars(E, D, P, vtype=GRB.BINARY, name="x")
    # Professor trabalhando no dia
    y = modelo.addVars(T, D, vtype=GRB.BINARY, name="y")
    # Sequência interna de eventos
    b = modelo.addVars(E, D, P, vtype=GRB.BINARY, name="b")
    g = modelo.addVars(E, D, P, vtype=GRB.BINARY, name="g")    # Aulas duplas
    # Número de duplas faltantes
    Gv = modelo.addVars(E, vtype=GRB.INTEGER, lb=0, name="G")
    z = modelo.addVars(T, D, P, P, vtype=GRB.BINARY,
                       name="z")  # Grafo de períodos ociosos

    # ------------------------------- Função Objetivo
    obj1 = quicksum(omega[t] * (n - m_ - 1) * z[t, d, m_, n]
                    for t in T for d in D for (m_, n) in U)
    obj2 = quicksum(gamma[t] * y[t, d] for t in T for d in D)
    obj3 = quicksum(delta[e] * Gv[e] for e in E)
    modelo.setObjective(obj1 + obj2 + obj3, GRB.MINIMIZE)

    # ------------------------------- Restrições

    # (2) Carga horária
    for e in E:
        modelo.addConstr(quicksum(x[e, d, p] for d in D for p in P) == R[e],
                         name=f"carga_{e}")

    # (3) Limite diário
    for e in E:
        for d in D:
            modelo.addConstr(quicksum(x[e, d, p] for p in P) <= L[e],
                             name=f"limiteDia_{e}_{d}")

    # (4) Disponibilidade de professores
    for e in E:
        for d in D:
            for p in P:
                professor = teacher_of_event[e]
                if e in V and d in V[e] and p in V[e][d] and V[e][d][p] == 0:
                    # Força a variável a 0 caso o professor esteja indisponível
                    x[e, d, p].ub = 0

    # (5) Professor único por período
    for t in T:
        for d in D:
            for p in P:
                modelo.addConstr(quicksum(x[e, d, p] for e in Et[t]) <= y[t, d],
                                 name=f"profSlot_{t}_{d}_{p}")

    # (6) Definição de y_td
    for t in T:
        for d in D:
            modelo.addConstr(quicksum(x[e, d, p] for e in Et[t] for p in P) >= y[t, d],
                             name=f"diaTrabalhado_{t}_{d}")

    # (7) Turma única por período
    for c in C:
        for d in D:
            for p in P:
                modelo.addConstr(quicksum(x[e, d, p] for e in Ec[c]) <= 1,
                                 name=f"turmaSlot_{c}_{d}_{p}")

    # (8)-(9) Sequência interna de um evento
    for e in E:
        for d in D:
            for p in P[1:]:
                modelo.addConstr(b[e, d, p] >= x[e, d, p] - x[e, d, p - 1],
                                 name=f"break_{e}_{d}_{p}")

    # (10)-(11) Aulas duplas
    for e in E:
        for (d, p) in SGe[e]:
            modelo.addConstr(g[e, d, p] <= x[e, d, p], name=f"g1_{e}_{d}_{p}")
            modelo.addConstr(g[e, d, p] <= x[e, d, p + 1],
                             name=f"g2_{e}_{d}_{p}")

    # (12) Número de duplas faltantes
    for e in M:
        modelo.addConstr(Gv[e] >= M[e] - quicksum(g[e, d, p] for (d, p) in SGe[e]),
                         name=f"duplasRestantes_{e}")

    # (13) Limite inferior de dias trabalhados
    for t in T:
        carga_t = sum(R[e] for e in Et[t])
        rhs = max(math.ceil(carga_t / len(P)),
                  max(math.ceil(R[e] / L[e]) for e in Et[t]))
        modelo.addConstr(quicksum(y[t, d]
                         for d in D) >= rhs, name=f"LBdias_{t}")

    # (14) ∑ z_{tdmn} = y_{td}  para m ≤ 3
    for t in T:
        for d in D:
            modelo.addConstr(
                quicksum(z[t, d, m, n] for (m, n) in Q if m <= 3) == y[t, d],
                name=f"z_igual_y_inicio_{t}_{d}"
            )

    # (15) ∑ z_{tdmn} ≤ y_{td}  para n ≥ 3
    for t in T:
        for d in D:
            modelo.addConstr(
                quicksum(z[t, d, m, n] for (m, n) in Q if n >= 3) <= y[t, d],
                name=f"z_menor_y_fim_{t}_{d}"
            )

    # (16) z_{tdpp} ≤ 1 + ∑ (x_{edp+1} - x_{edp})
    for t in T:
        for d in D:
            for p in P[:-1]:  # p ∈ P'
                modelo.addConstr(
                    z[t, d, p, p] <= 1 +
                    quicksum(x[e, d, p + 1] - x[e, d, p] for e in Et[t]),
                    name=f"z16_{t}_{d}_{p}"
                )

    # (17) z_{td(m,m+1)} ≤ 1 - ∑ x_{edm}
    for t in T:
        for d in D:
            for (m, n) in U:
                if n == m + 1:
                    modelo.addConstr(
                        z[t, d, m, n] <= 1 -
                        quicksum(x[e, d, m] for e in Et[t]),
                        name=f"z17_{t}_{d}_{m}_{n}"
                    )

    # (18) z_{tdmn} ≤ ∑ x_{edn}
    for t in T:
        for d in D:
            for (m, n) in U:
                modelo.addConstr(
                    z[t, d, m, n] <= quicksum(x[e, d, n] for e in Et[t]),
                    name=f"z18_{t}_{d}_{m}_{n}"
                )
   

    return modelo

# Célula 3: Carregar dados e construir modelo
json_data = carregar_json("final_output_Dorneles.json")
modelo = constroi_modelo(json_data)
modelo.write('modelo_dorneles.lp')

# Célula 4: Configurar e resolver o modelo
modelo.setParam("TimeLimit", 1800)  # 10 minutos por padrão
modelo.setParam("MIPGap", 0.0001)    # 1% de gap
modelo.optimize()



Set parameter TimeLimit to value 1800
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13650HX, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  1800

Optimize a model with 1370 rows, 2636 columns and 6856 nonzeros
Model fingerprint: 0x7c46231e
Variable types: 0 continuous, 2636 integer (2615 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Found heuristic solution: objective 342.0000000
Presolve removed 936 rows and 2071 columns
Set parameter MIPGap to value 0.0001
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13650HX, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20

In [ ]:

# Célula 5: Pós-processamento e geração do Excel
if modelo.Status == GRB.OPTIMAL or modelo.Status == GRB.TIME_LIMIT:
    print(f"Melhor solução encontrada: {modelo.ObjVal:.0f}")
    modelo.write('solucao_dorneles.mst')

    # --- Chamada dinâmica do módulo solucao_grade_horaria.py ---
    import importlib.util
    import sys
    import os

    script_dir = os.path.dirname(os.path.abspath("hstt_model_v2.ipynb"))
    if script_dir not in sys.path:
        sys.path.append(script_dir)

    spec = importlib.util.spec_from_file_location("solucao_grade_horaria", os.path.join(script_dir, "solucao_grade_horaria.py"))
    solucao_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(solucao_module)

    # Chamada da função do módulo
    solucao_module.solucao_grade_horaria()
    solucao_module.calcular_metricas_e_adicionar_excel()

else:
    print("Modelo infactível ou outra condição.")

Modelo infactível ou outra condição.


1. Calcular o valor da função objetivo para a solução da imagem.

#Artigo
Dias trabalhados: 21*9
Janelas: 3*3
Aulas duplas não atendidas: 2*1

In [62]:
21*9+3*3+2*1

200

#Imagem
Dias trabalhados: 20*9
Janelas: 4*3
Aulas duplas não atendidas: 2*1

In [63]:
20*9+4*3+2*1

194

 
2. O artigo apresenta um valor ótimo para a função objetivo (A), a solução da imagem tem um valor de função objetivo (I) e você obteve um valor ótimo (S) para a função objetivo na sua implementação. Explique o que pode ter acontecido para cenários:
    a. A>I>S
    b. A>S>I
    c. S>A>I
    d. S>I>A
    e. I>A>S
    f. I>S>A



## c. S > A > I  (204 > 200 > 194)
A solução encontrada foi a pior, a do artigo intermediária, e a da imagem a melhor. Isso pode indicar:
- Implementação incorreta. Ou menos eficiente.
- A solução da imagem pode não respeitar todas as restrições do modelo.
- O artigo pode ter relaxado algumas restrições ou usado heurísticas diferentes.

 
3. Force no seu código a solução apresentada na imagem e identifique qual(is) restrição(ões) (caso aconteça) são violadas.
 

In [64]:
import json
import math

def carregar_json(caminho_arquivo):
    """Função para carregar o arquivo JSON e retornar os dados."""
    with open(caminho_arquivo, "r") as f:
        data = json.load(f)
    return data

def constroi_modelo(json_data):
    """Constrói o modelo HSTT a partir dos dados carregados do JSON."""
    # Dados do JSON
    D = json_data["D"]  # Dias
    P = json_data["P"]  # Períodos
    T = json_data["T"]  # Professores
    C = json_data["C"]  # Turmas
    E = json_data["E"]  # Eventos
    R = json_data["R"]  # Carga horária de cada evento
    L = json_data["L"]  # Limite diário de aulas por evento
    M = json_data["M"]  # Número mínimo de aulas duplas por evento
    gamma = json_data["gamma"]  # Custo dia trabalhado por professor
    omega = json_data["omega"]  # Custo intervalo ocioso
    delta = json_data["delta"]  # Custo aula dupla não atendida
    teacher_of_event = json_data["teacher_of_event"]  # Mapeamento de professor para evento
    class_of_event = json_data["class_of_event"]  # Mapeamento de turma para evento
    V = json_data["V"]  # Disponibilidade dos professores

    # Conjuntos derivados
    Et = {t: [e for e in E if teacher_of_event[e] == t] for t in T}  # Eventos por professor
    Ec = {c: [e for e in E if class_of_event[e] == c] for c in C}  # Eventos por turma

    # SGe – slots onde e pode iniciar aula dupla
    SGe = {}
    for e in E:
        SGe[e] = []
        for d in D:
            for p in P[:-1]:  # Excluindo o último período
                if (
                    e in V
                    and d in V[e]
                    and p in V[e][d]
                    and (p + 1) in V[e][d]
                    and V[e][d][p]
                    and V[e][d][p + 1]
                ):
                    # Verificando se o evento e, dia d, e períodos p e p+1 existem e são válidos
                    SGe[e].append((d, p))

    # U e Q – arcos para o grafo de períodos ociosos
    U, Q = [], []
    for m in P[:-2]:  # Excluindo os dois últimos períodos
        for n in P:
            if n >= m:
                Q.append((m, n))
            if n >= m + 2:
                U.append((m, n))

    # Construção do modelo
    modelo = LpProblem("HSTT_Model", LpMinimize)

    # ------------------------------- Variáveis
    x = LpVariable.dicts("x", (E, D, P), cat=LpBinary)  # Variáveis de alocação de eventos
    y = LpVariable.dicts("y", (T, D), cat=LpBinary)  # Professor trabalhando no dia
    b = LpVariable.dicts("b", (E, D, P), cat=LpBinary)  # Sequência interna de eventos
    g = LpVariable.dicts("g", (E, D, P), cat=LpBinary)  # Aulas duplas
    Gv = LpVariable.dicts("G", E, lowBound=0, cat=LpInteger)  # Número de duplas faltantes
    z = LpVariable.dicts("z", (T, D, P, P), cat=LpBinary)  # Grafo de períodos ociosos

    # ------------------------------- Função Objetivo
    obj1 = lpSum(omega[t] * (n - m_ - 1) * z[t][d][m_][n] for t in T for d in D for (m_, n) in U)  # Corrigindo a fórmula
    obj2 = lpSum(gamma[t] * y[t][d] for t in T for d in D)
    obj3 = lpSum(delta[e] * Gv[e] for e in E)
    modelo += obj1 + obj2 + obj3

    # ------------------------------- Restrições

    # (2) Carga horária
    for e in E:
        modelo += lpSum(x[e][d][p] for d in D for p in P) == R[e], f"carga_{e}"

    # (3) Limite diário
    for e in E:
        for d in D:
            modelo += lpSum(x[e][d][p] for p in P) <= L[e], f"limiteDia_{e}_{d}"

    # (4) Disponibilidade de professores
    for e in E:
        for d in D:
            for p in P:
                professor = teacher_of_event[e]
                if e in V and d in V[e] and p in V[e][d] and V[e][d][p] == 0:
                    modelo += x[e][d][p] == 0, f"disponibilidade_{e}_{d}_{p}"

    # (5) Professor único por período
    for t in T:
        for d in D:
            for p in P:
                modelo += lpSum(x[e][d][p] for e in Et[t]) <= y[t][d], f"profSlot_{t}_{d}_{p}"

    # (6) Definição de y_td
    for t in T:
        for d in D:
            modelo += lpSum(x[e][d][p] for e in Et[t] for p in P) >= y[t][d], f"diaTrabalhado_{t}_{d}"

    # (7) Turma única por período
    for c in C:
        for d in D:
            for p in P:
                modelo += lpSum(x[e][d][p] for e in Ec[c]) <= 1, f"turmaSlot_{c}_{d}_{p}"

    # (8) Sequência interna de um evento
    for e in E:
        for d in D:
            for p in P[1:]:
                modelo += b[e][d][p] >= x[e][d][p] - x[e][d][p - 1], f"break_{e}_{d}_{p}"

    # (9) Continuidade das aulas
    for e in E:
        for d in D:
            for p in P[:-1]:
                modelo += b[e][d][p] + x[e][d][p+1] <= 1, f"continuidade_{e}_{d}_{p}"

    # (10)-(11) Aulas duplas (já existentes, mas reforçadas aqui)
    for e in E:
        for d, p in SGe[e]:
            modelo += g[e][d][p] <= x[e][d][p]
            modelo += g[e][d][p] <= x[e][d][p + 1]
            modelo += g[e][d][p] >= x[e][d][p] + x[e][d][p + 1] - 1

    # (12) Número de duplas faltantes
    for e in M:
        modelo += Gv[e] >= M[e] - lpSum(g[e][d][p] for (d, p) in SGe[e]), f"duplasRestantes_{e}"


    # (13) Limite inferior de dias trabalhados
    for t in T:
        carga_t = sum(R[e] for e in Et[t])
        rhs = max(math.ceil(carga_t / len(P)), max(math.ceil(R[e] / L[e]) for e in Et[t]))
        modelo += lpSum(y[t][d] for d in D) >= rhs, f"LBdias_{t}"
    

    # (14) Ativação de janelas (z) 
    for t in T:
        for d in D:
            for (m, n) in U:
                modelo += z[t][d][m][n] >= (
                    lpSum(x[e][d][m] for e in Et[t]) +
                    lpSum(x[e][d][n] for e in Et[t]) -
                    lpSum(x[e][d][k] for e in Et[t] for k in range(m + 1, n))
                ) - 1, f"ativar_janela_{t}_{d}_{m}_{n}"

    # (14) Ativação de janelas (z <= y)
    for t in T:
        for d in D:
            for (m, n) in U:
                modelo += z[t][d][m][n] <= y[t][d], f"z_igual_y_{t}_{d}_{m}_{n}"
    
    # (15) Limite superior de aulas duplas
    for t in T:
        for d in D:
            for (m, n) in U:
                modelo += z[t][d][m][n] <= 1 - quicksum(x[e][d][m] for e in Et[t]), f"z_inicio_{t}_{d}_{m}_{n}"
                modelo += z[t][d][m][n] <= 1 - quicksum(x[e][d][n] for e in Et[t]), f"z_fim_{t}_{d}_{m}_{n}"

    # (15) Ativação de janelas (complementares)
    for t in T:
        for d in D:
            modelo += lpSum(x[e][d][p] for e in Et[t] for p in P) >= 2 * y[t][d], f"minimo_aulas_dia_{t}_{d}"

    # (16) Limite superior de dias trabalhados (exemplo: no máximo 4 dias)
    for t in T:
        modelo += lpSum(y[t][d] for d in D) <= 3, f"max_dias_trabalho_{t}"
    
    # (17) Limite superior de aulas duplas
    for e in json_data["E"]:
        modelo.addConstr(
            quicksum(g[e, d, p] for (d, p) in SGe[e]) >= M[e] - Gv[e],
            name=f"duplas_minimas_{e}"
        )

    # (17) Restrição de soma das janelas
    for t in T:
        for d in D:
            for (m, n) in Q:
                modelo += z[t][d][m][n] + 1 <= quicksum(x[e][d][n] for e in Et[t]), f"z_fim2_{t}_{d}_{m}_{n}"

    # (18) Evitar sobreposição de duplas entre eventos do mesmo professor/turma
    for t in T:
        for d in D:
            for p in P[:-1]:
                modelo.addConstr(
                    quicksum(g[e, d, p] for e in Et[t]) <= 1,
                    name=f"dupla_professor_unica_{t}_{d}_{p}"
                )
    for c in C:
        for d in D:
            for p in P[:-1]:
                modelo.addConstr(
                    quicksum(g[e, d, p] for e in Ec[c]) <= 1,
                    name=f"dupla_turma_unica_{c}_{d}_{p}"
                )

    return modelo


from gurobipy import Model, GRB, quicksum
import importlib.util
import json

def carregar_json(caminho_arquivo):
    with open(caminho_arquivo, "r") as f:
        data = json.load(f)
    return data

def constroi_modelo(json_data):
    D = json_data["D"]
    P = json_data["P"]
    T = json_data["T"]
    C = json_data["C"]
    E = json_data["E"]
    R = json_data["R"]
    L = json_data["L"]
    M = json_data["M"]
    gamma = json_data["gamma"]
    omega = json_data["omega"]
    delta = json_data["delta"]
    teacher_of_event = json_data["teacher_of_event"]
    class_of_event = json_data["class_of_event"]
    V = json_data["V"]

    Et = {t: [e for e in E if teacher_of_event[e] == t] for t in T}
    Ec = {c: [e for e in E if class_of_event[e] == c] for c in C}

    SGe = {}
    for e in E:
        SGe[e] = []
        for d in D:
            for p in P[:-1]:
                if (
                    e in V and d in V[e] and p in V[e][d] and (p + 1) in V[e][d]
                    and V[e][d][p] and V[e][d][p + 1]
                ):
                    SGe[e].append((d, p))

    U = []
    for m in P[:-2]:
        for n in P:
            if n >= m + 2:
                U.append((m, n))

    # Modelo Gurobi
    modelo = Model("HSTT_Model")
    modelo.Params.OutputFlag = 1

    # Variáveis
    x = modelo.addVars(E, D, P, vtype=GRB.BINARY, name="x")
    y = modelo.addVars(T, D, vtype=GRB.BINARY, name="y")
    b = modelo.addVars(E, D, P, vtype=GRB.BINARY, name="b")
    g = modelo.addVars(E, D, P, vtype=GRB.BINARY, name="g")
    Gv = modelo.addVars(E, vtype=GRB.INTEGER, lb=0, name="G")
    z = modelo.addVars(T, D, P, P, vtype=GRB.BINARY, name="z")

    # Função objetivo
    obj1 = quicksum(omega[t] * (n - m - 1) * z[t, d, m, n] for t in T for d in D for (m, n) in U)
    obj2 = quicksum(gamma[t] * y[t, d] for t in T for d in D)
    obj3 = quicksum(delta[e] * Gv[e] for e in E)
    modelo.setObjective(obj1 + obj2 + obj3, GRB.MINIMIZE)

    # Restrições
    for e in E:
        modelo.addConstr(quicksum(x[e, d, p] for d in D for p in P) == R[e], name=f"carga_{e}")

    for e in E:
        for d in D:
            modelo.addConstr(quicksum(x[e, d, p] for p in P) <= L[e], name=f"limiteDia_{e}_{d}")

    for e in E:
        for d in D:
            for p in P:
                if e in V and d in V[e] and p in V[e][d] and V[e][d][p] == 0:
                    modelo.addConstr(x[e, d, p] == 0, name=f"disp_{e}_{d}_{p}")

    for t in T:
        for d in D:
            for p in P:
                modelo.addConstr(quicksum(x[e, d, p] for e in Et[t]) <= y[t, d], name=f"profSlot_{t}_{d}_{p}")

    for t in T:
        for d in D:
            modelo.addConstr(quicksum(x[e, d, p] for e in Et[t] for p in P) >= y[t, d], name=f"diaTrabalhado_{t}_{d}")

    for c in C:
        for d in D:
            for p in P:
                modelo.addConstr(quicksum(x[e, d, p] for e in Ec[c]) <= 1, name=f"turmaSlot_{c}_{d}_{p}")

    for e in E:
        for d in D:
            for p in P[1:]:
                modelo.addConstr(b[e, d, p] >= x[e, d, p] - x[e, d, p - 1], name=f"break_{e}_{d}_{p}")

    for e in E:
        for d, p in SGe[e]:
            modelo.addConstr(g[e, d, p] <= x[e, d, p], name=f"g1_{e}_{d}_{p}")
            modelo.addConstr(g[e, d, p] <= x[e, d, p + 1], name=f"g2_{e}_{d}_{p}")
            modelo.addConstr(g[e, d, p] >= x[e, d, p] + x[e, d, p + 1] - 1, name=f"g3_{e}_{d}_{p}")

    for e in M:
        modelo.addConstr(Gv[e] >= M[e] - quicksum(g[e, d, p] for (d, p) in SGe[e]), name=f"duplasRestantes_{e}")

    import math
    for t in T:
        carga_t = sum(R[e] for e in Et[t])
        rhs = max(math.ceil(carga_t / len(P)), max(math.ceil(R[e] / L[e]) for e in Et[t]))
        modelo.addConstr(quicksum(y[t, d] for d in D) >= rhs, name=f"LBdias_{t}")

    for t in T:
        for d in D:
            for (m, n) in U:
                modelo.addConstr(
                    z[t, d, m, n] >= (
                        quicksum(x[e, d, m] for e in Et[t]) +
                        quicksum(x[e, d, n] for e in Et[t]) -
                        quicksum(x[e, d, k] for e in Et[t] for k in range(m + 1, n))
                    ) - 1, name=f"ativar_janela_{t}_{d}_{m}_{n}"
                )

    for t in T:
        for d in D:
            modelo.addConstr(quicksum(x[e, d, p] for e in Et[t] for p in P) >= 2 * y[t, d], name=f"minimo_aulas_dia_{t}_{d}")

    for t in T:
        modelo.addConstr(quicksum(y[t, d] for d in D) <= 3, name=f"max_dias_trabalho_{t}")

    return modelo, x

# Carregar os dados do JSON
json_data = carregar_json("final_output_Dorneles.json")

# Construir o modelo
modelo, x = constroi_modelo(json_data)

# Escrever o modelo em arquivo LP (opcional)
modelo.write("modelo_dorneles.lp")

# Configurações do modelo
modelo.Params.TimeLimit = 300  # 5 minutos
modelo.Params.MIPGap = 0.01   # 1% de gap

# Resolver o modelo
modelo.optimize()

import csv

with open("tabela_alocacao.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["Evento", "Professor", "Turma", "Dia", "Período"])

    for e in json_data["E"]:
        prof = json_data["teacher_of_event"][e]
        turma = json_data["class_of_event"][e]
        for d in json_data["D"]:
            for p in json_data["P"]:
                var = x[e, d, p]
                if var.X > 0.5:
                    writer.writerow([e, prof, turma, d, p])

# Verificar o status da solução
if modelo.Status == GRB.OPTIMAL or modelo.Status == GRB.TIME_LIMIT:
    print(f"Ótimo encontrado: {modelo.ObjVal:.0f}")

    # Importar dinamicamente o módulo solucao_grade_horaria.py
    spec = importlib.util.spec_from_file_location("solucao_grade_horaria", "solucao_grade_horaria.py")
    solucao_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(solucao_module)

    # Executar as funções
    solucao_module.solucao_grade_horaria()
    solucao_module.calcular_metricas_e_adicionar_excel()

    # ---- MÉTRICAS SOLICITADAS ----
    # Dias de trabalho distintos
    dias_trabalho = set()
    for t in json_data["T"]:
        for d in json_data["D"]:
            if modelo.getVarByName(f"y[{t},{d}]").X > 0.5:
                dias_trabalho.add((t, d))
    print(f"Dias de trabalho distintos: {len(dias_trabalho)}")

    # Total de janelas
    total_janelas = 0
    for t in json_data["T"]:
        for d in json_data["D"]:
            for m in json_data["P"][:-2]:
                for n in json_data["P"]:
                    if n >= m + 2:
                        var = modelo.getVarByName(f"z[{t},{d},{m},{n}]")
                        if var and var.X > 0.5:
                            total_janelas += 1
    print(f"Total de janelas: {total_janelas}")

    # Aulas duplas não atendidas
    duplas_nao_atendidas = 0
    for e in json_data["E"]:
        var = modelo.getVarByName(f"G[{e}]")
        if var:
            duplas_nao_atendidas += int(round(var.X))
    print(f"Aulas duplas não atendidas: {duplas_nao_atendidas}")
    
    import csv
    # Salva as aulas duplas não atendidas por evento para uso no Excel
    with open("duplas_nao_atendidas_por_evento.csv", "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["Evento", "DuplasNaoAtendidas"])
        for e in json_data["E"]:
            var = modelo.getVarByName(f"G[{e}]")
            if var:
                writer.writerow([e, int(round(var.X))])

else:
    print("Modelo não ótimo ou infactível.")

Set parameter OutputFlag to value 1
Set parameter TimeLimit to value 300
Set parameter MIPGap to value 0.01
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13650HX, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  300
MIPGap  0.01

Optimize a model with 1178 rows, 2636 columns and 7341 nonzeros
Model fingerprint: 0x8e35487b
Variable types: 0 continuous, 2636 integer (2615 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]
Presolve removed 501 rows and 1831 columns
Presolve time: 0.01s
Presolved: 677 rows, 805 columns, 5410 nonzeros
Variable types: 0 continuous, 805 integer (805 binary)

Root relaxation: objective 1.890000e+02, 725 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current 

KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\cheri\anaconda3\Lib\site-packages\ipykernel\iostream.py", line 624, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 59445 32326  201.00000   49   51  204.00000  189.00000  7.35%  52.4   20s
 74486 40085  193.57143   38  123  204.00000  189.00000  7.35%  51.9   25s
 101645 54228  198.75000   55   65  204.00000  189.00000  7.35%  49.4   30s
 115056 61828 infeasible   59       204.00000  189.00000  7.35%  49.1   35s
 127476 67455  192.50000   49   64  204.00000  189.00000  7.35%  48.6   40s
 146509 78836  189.00000   38  107  204.00000  189.00000  7.35%  47.7   45s
 170039 91039     cutoff   56       204.00000  189.00000  7.35%  46.6   50s
 199216 105900  201.00000   62   56  204.00000  189.00000  7.35%  45.4   55s


KeyboardInterrupt: 

Exception ignored in: 'gurobipy._core.logcallbackstub'
Traceback (most recent call last):
  File "c:\Users\cheri\anaconda3\Lib\site-packages\ipykernel\iostream.py", line 624, in write
    def write(self, string: str) -> Optional[int]:  # type:ignore[override]

KeyboardInterrupt: 


 251261 133018     cutoff   63       204.00000  189.00000  7.35%  44.5   65s
 279613 148574  199.50000   48   53  204.00000  189.00000  7.35%  44.5   70s
 307886 162927     cutoff   59       204.00000  189.00000  7.35%  44.1   75s
 328073 173616  196.50000   54   45  204.00000  189.00000  7.35%  43.9   80s
 344133 182228  190.50000   59   75  204.00000  189.00000  7.35%  43.8   85s
 355326 196465  201.00000   56   63  204.00000  189.00000  7.35%  43.7   94s
 374709 197647  196.50000   66   52  204.00000  189.00000  7.35%  43.5   96s
 392731 206240  198.00000   45   73  204.00000  189.00000  7.35%  43.1  100s
 407313 213224  195.00000   47   59  204.00000  189.00000  7.35%  42.9  105s
 421396 220879  199.50000   49   89  204.00000  189.00000  7.35%  42.8  110s
 443282 233285  198.21429   54  109  204.00000  189.00000  7.35%  42.5  115s
 470424 247547  189.37500   36   84  204.00000  189.00000  7.35%  42.2  120s
 497816 261338  193.50000   57   53  204.00000  189.00000  7.35%  42.0  125s

c:\Users\cheri\iCloudDrive\UFPR\MNUM7095 - Modelos de Timetabling\class3\solucao_grade_horaria.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Turma"] = df["Turma"].replace(turma_map).astype("Int64")
c:\Users\cheri\iCloudDrive\UFPR\MNUM7095 - Modelos de Timetabling\class3\solucao_grade_horaria.py:242: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  wb.save("solucao_grade_horaria.xlsx")


In [80]:
# Após definir o modelo e as variáveis, force a solução da imagem:
import pandas as pd

# Carrega a solução da imagem (igual ao formato gerado por solucao_grade_horaria.py)
df_img = pd.read_excel("solucao_grade_horaria_imagem.xlsx", sheet_name="Grade Horária", index_col=0)

# Para cada professor, dia, período, força x[e][d][p] conforme a solução da imagem
# Como não temos o evento na planilha, vamos forçar apenas as variáveis x para os eventos possíveis daquele professor/turma
for prof in df_img.index:
    for d in range(5):
        for p in range(5):
            col = df_img.columns[d * 5 + p]
            val = df_img.loc[prof, col]
            if pd.notna(val):
                turma = int(val)
                # Para cada evento desse professor e turma
                for e in [ev for ev in E if teacher_of_event[ev] == prof and class_of_event[ev] == f"c{turma}"]:
                    # Força x[e][d][p] = 1
                    x[e, d, p].lb = 1
                    x[e, d, p].ub = 1
            else:
                # Para todos eventos possíveis desse professor, força x=0
                for e in [ev for ev in E if teacher_of_event[ev] == prof]:
                    x[e, d, p].ub = 0

# Otimiza o modelo forçando a solução da imagem
modelo.optimize()

# Após otimizar, verifica restrições violadas
violadas = []
for constr in modelo.getConstrs():
    slack = constr.getAttr("Slack")
    if abs(slack) > 1e-5 and constr.getAttr("Sense") == '=':
        violadas.append((constr.ConstrName, slack))
    elif slack < -1e-5 and constr.getAttr("Sense") == '<':
        violadas.append((constr.ConstrName, slack))
    elif slack > 1e-5 and constr.getAttr("Sense") == '>':
        violadas.append((constr.ConstrName, slack))

if violadas:
    print("Restrições violadas na solução da imagem:")
    for nome, slack in violadas:
        print(f"{nome}: slack={slack}")
else:
    print("Nenhuma restrição violada na solução da imagem.")

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13650HX, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Non-default parameters:
TimeLimit  300
MIPGap  0.01

Optimize a model with 1178 rows, 2636 columns and 7341 nonzeros
Model fingerprint: 0x3c0abb0f
Variable types: 0 continuous, 2636 integer (2615 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 9e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+00]

MIP start from previous solve did not produce a new incumbent solution

Presolve removed 824 rows and 2236 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 20 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -


AttributeError: Unable to retrieve attribute 'Slack'

 
4. Faça testes nos seus códigos alterando os pesos na função objetivo e avalie o impacto na solução (não em custo da função objetivo, mas nas características delas. Por exemplo, números de dias trabalhados pelos professores, quantidade de janelas dos professores, etc.

Em meus testes mudar os pesos não fez muita diferença pois o maior problema que tive foi em conseguir levar em consideração os dias de aulas duplas não atendidas.
